In [1]:
from collections import defaultdict
import re
from copy import deepcopy
from itertools import combinations, count
import heapq

class State: 
    def __init__(self): 
        # My input
        self.floors = [["GPR", "MPR"], 
                  ["GCO", "GCU", "GRU", "GPL"],
                  ["MCO", "MCU", "MRU", "MPL"], 
                  []]

# my prev input
#         self.floors = [["GPO", "GTH", "MTH", "GPR", "GRU", "MRU", "GCU", "MCU"], 
#                   ["MPO", "MPR"],
#                   [],
#                   []]

        # Part 2
  #      self.floors[0].extend(["GE", "ME", "GD", "MD"])



#         self.floors = [["MH", "ML"], 
#                   ["GH"],
#                   ["GL"], 
#                   []]

        self.elevator_pos = 0
        
    def is_goal(self): 
        return len(self.floors[0]) == 0 and len(self.floors[1]) == 0 and len(self.floors[2]) == 0
    
    def state_representation(self): 
        repres = f"{self.elevator_pos}"
        for floor in self.floors: 
            chips = [x for x in floor if x.startswith("M")]
            gens = [x for x in floor if x.startswith("G")]            
            repres += f"({len(chips)}, {len(gens)})"
        return repres
    
    def is_valid(self): 
        for floor in self.floors: 
            single_chips = list()
            all_generators = list()            
            single_generators = list()
            
            for item in floor: 
                if item.startswith("G"): 
                    all_generators.append(item)
                    matching_chip = item.replace("G","M")
                    if matching_chip not in floor: 
                        single_generators.append(item)
                elif item.startswith("M"): 
                    matching_generator = item.replace("M","G")
                    if matching_generator not in floor: 
                        single_chips.append(item)
            if len(single_chips) > 0 and len(all_generators) > 0: 
                return False

        return True
    
    def __repr__(self): 
        return "<< " + str(self.elevator_pos) + " == " + str(self.floors) + ">>"
    def expand(self): 
        newstates = list()
        
        for delta_elevator in [-1, 1]: 
            newpos = self.elevator_pos + delta_elevator
            if 0 <= newpos < len(self.floors): 
                # Bring single object
                for bringobj in self.floors[self.elevator_pos]: 
                    newstate = deepcopy(self)
                    newstate.floors[self.elevator_pos].remove(bringobj)
                    newstate.floors[newpos].append(bringobj)
                    newstate.elevator_pos = newpos
                    newstates.append(newstate)
                    
                # Bring two objects
                for bringobj1, bringobj2 in combinations(self.floors[self.elevator_pos], r=2): 
                    l1, l2 = bringobj1[0], bringobj2[0]
                    if l1 != l2: 
                        if bringobj1[1:] != bringobj2[1:]: 
                            continue
                    newstate = deepcopy(self)
                    newstate.floors[self.elevator_pos].remove(bringobj1)
                    newstate.floors[self.elevator_pos].remove(bringobj2)                    
                    newstate.floors[newpos].append(bringobj1)
                    newstate.floors[newpos].append(bringobj2)                    
                    newstate.elevator_pos = newpos
                    newstates.append(newstate)
        newstates = [state for state in newstates if state.is_valid()]
        return newstates
    
    
    
a = State()
counter = count()

num_steps = defaultdict(lambda: 999999)
num_steps[str(a)] = 0
to_expand = [(0, a)]

while to_expand: 
    steps_so_far, state = to_expand.pop(0)

    if state.is_goal(): 
        print("REACHED GOAL")
        print(steps_so_far, state)
        break
    newsteps = steps_so_far + 1
    for expansion in state.expand(): 
        if newsteps < num_steps[expansion.state_representation()]:
            num_steps[expansion.state_representation()] = newsteps
            to_expand.append((newsteps, expansion))
    
    


REACHED GOAL
33 << 3 == [[], [], [], ['GRU', 'GPL', 'MRU', 'MPL', 'GCO', 'GPR', 'MCO', 'MPR', 'MCU', 'GCU']]>>
